# 再トレーニングされた YOLO モデルをテストする

モデルを再トレーニングしたので、それを 'test' 画像に対して使用できるようになりました。

In [ ]:
# このラボ用に設計されたワークベンチ イメージを使用しなかった場合は、コメントを解除して次の行を実行して、必要なパッケージをインストールできます。
# !pip install --no-cache-dir --no-dependencies -r requirements.txt

from ultralytics import YOLO
from PIL import Image
import numpy as np
import cv2
from matplotlib import pyplot as plt

import remote_infer

前のノートブックでは、モデルを再トレーニングする方法について説明しました。これは GPU の効率性を必要とする時間のかかるタスクであるため、ワークショップ中に実際に実行することはできません。

そのため、便宜上、ONNX 形式でエクスポートされた、再トレーニング済みのモデルを利用します。

In [ ]:
# 新しくトレーニングされたモデルを取得します。

model = YOLO("https://rhods-public.s3.amazonaws.com/demo-models/ic-models/accident/accident_detect.onnx", task="detect")

In [ ]:
# 自動車事故の画像に対してモデルをテストします。
image_path = "images/carImage3.jpg" # We know that Image3 represents a severe accident with over 86% certainty.

_, scale, original_image = remote_infer.preprocess(image_path)

original_image: np.ndarray = cv2.imread(image_path)
blob = cv2.dnn.blobFromImage(original_image, size=(640, 640), swapRB=False)
blob = np.ascontiguousarray(blob[0].transpose((1,2,0)))
results = model.predict(blob)

In [ ]:
# 結果からすべての情報を抽出します。（タイプ、境界ボックス、確率）

detections = []
result = results[0]
for box in result.boxes:
    class_id = int(box.cls.item())
    score = box.conf.item()
    unscaled_cords = box.xyxy.squeeze().tolist()
    cords = [round(unscaled_cords[0] * scale[1]), round(unscaled_cords[1] * scale[0]), round(unscaled_cords[2] * scale[1]), round(unscaled_cords[3] * scale[0])]
    detection = {
        'class_id': class_id,
        'class_name': result.names[class_id],
        'confidence': score,
        'box': cords,
        'scale': scale}
    detections.append(detection)
    print(detection)
    remote_infer.draw_bounding_box(original_image, class_id, score, cords[0], cords[1], cords[2], cords[3])

In [ ]:
# 画像上に、ボックス、クラス名、確率（モデルが検出についてどの程度確信しているか） を描画します。

img = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
fig = plt.gcf()
fig.set_size_inches(6, 3)
plt.axis('off')
plt.imshow(img)

 'carImage3.jpg' を調べると、再トレーニングされた YOLO モデルは 86% の信頼度で '自動車事故' を正確に予測します。次に、自動車事故の周囲にボックスが描かれ、 `severe 0.86` というラベルが付けられます。

事故の重大度を検出できるモデルができたので、予測関数を作成し、ModelMesh を使用してモデルを提供しましょう。

**その方法については、ラボガイドに戻ってください。**

**ノートブック `04-05-model-serving.ipynb` はまだ開かないでください。**